# Clustering Neighbourhoods in Toronto

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Download information from Wikipedia page  

In [3]:
uri = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(uri)

soup = BeautifulSoup(html_data.content, 'html.parser')
soup.prettify()

tables = soup.body.findAll("table")

and parse it with Beautiful soap

In [4]:
table_contents = []
table = tables[0]
for td in table.tbody.find_all("td"):
    if td.span.text == "Not assigned":
        pass
    else:
        cell = {}
        cell['PostalCode'] = td.p.text[:3]
        cell['Borough'] = td.span.text.split('(')[0]
        cell['Neighborhood'] = (((((td.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
        table_contents.append(cell)

df=pd.DataFrame(table_contents)

df.shape

(103, 3)

Retrieve Latitude and Longitude information for each postal code from the CSV file

In [46]:
geo_c = pd.read_csv("Geospatial_Coordinates.csv")
latitudes = []
longitudes = []
for postalcode in df['PostalCode']:
    g = geo_c.loc[geo_c['Postal Code'] == postalcode]
    
    latitudes.append(g.iat[0, 1])
    longitudes.append(g.iat[0, 2])

#print(latitudes.dtype)    
df['Latitude'] = latitudes
df['Longitude'] = longitudes

df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
